In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=[{"role":"user","content":"Write a limerick about the wonders of GPU computing."}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



In [ ]:
import torch 


raw_labels = torch.load("/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/meta/llama-3.1-8b-instruct/flan_v2/total_output_labels.pt")



In [ ]:
import collections
import matplotlib.pyplot as plt
import torch 

model_name="meta/llama-3.1-8b-instruct"
dataset_name = 'cot'

path = "/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/"
raw_labels = torch.load(path + f"{model_name}/{dataset_name}/total_output_labels.pt")

# 统计 labels 列表中的值
label_counts = collections.Counter(raw_labels)

# 打印统计结果
sorted_label_counts = dict(sorted(label_counts.items()))
print("Sorted Label counts:", sorted_label_counts)

# 绘制直方图
labels, counts = zip(*sorted_label_counts.items())

plt.bar(labels, counts, align='center')
plt.xlabel('Labels')
plt.ylabel('Label Counts')
plt.title(f'Labeling Model: {model_name};; Dataset: {dataset_name}; Label Distribution')
plt.xticks(range(min(labels), max(labels) + 1))

plt.text(0.05, 0.95, f'Total Labels: {len(raw_labels)}', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)


plt.show()



In [ ]:
from datasets import load_dataset


dataset_name = 'flan_v2'

data = load_dataset('parquet', data_files=f'data_refine/tulu_split_parquet/{dataset_name}.parquet', split=None, cache_dir=None)

dialogs = data['train']

In [ ]:
data

In [ ]:
import os
import torch

# 文件夹路径
folder_path = "/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/meta/llama-3.1-8b-instruct/cot"
incorrect_files = []

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.startswith("output_labels_") and filename.endswith(".pt"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 加载文件内容
            labels = torch.load(file_path)
            
            # 检查文件长度是否等于 1024
            if len(labels) != 1024:
                incorrect_files.append(filename)
                print(f"{filename}: {len(labels)} labels (incorrect length)")
            # else:
                # print(f"{filename}: {len(labels)} labels (correct length)")
        
        except Exception as e:
            print(f"Error loading {filename}: {e}")


In [ ]:
from huggingface_hub import HfApi, upload_file, upload_folder

# 第一步：创建 Hugging Face API 实例
api = HfApi()

# 第二步：创建数据集仓库
# 将 'username' 替换为你的 Hugging Face 用户名，'dataset_name' 替换为你想要创建的数据集名称
repo_id = "jlpang888/LLM_Data_selection"  
# api.create_repo(repo_id=repo_id, repo_type="dataset")

# 第三步：上传文件
# 指定本地文件路径和上传路径

file_path = "model_finetune_cluster/new_train_data/mistralai/Mistral-7B-Instruct-v0.3/all_train/filtered-cured_dataset.json"  # 本地数据文件路径
path_in_repo = "mistral_labeling_filtered-cured.json"            # 上传到 Hugging Face 数据集中显示的路径和文件名
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo=path_in_repo,
    repo_id=repo_id,
    repo_type="dataset"
)

# 如果你想上传一个文件夹而不是单个文件，可以使用 upload_folder 函数：
# folder_path = "model_finetune_cluster/new_train_data"  # 你的文件夹路径
# upload_folder(
#     folder_path=folder_path,
#     repo_id=repo_id,  # 上传到相同的 repo_id
#     repo_type="dataset"
# )

print(f"数据集已成功上传到 Hugging Face: https://huggingface.co/datasets/{repo_id}")


In [2]:
from huggingface_hub import HfApi, upload_folder
from transformers import AutoModelForCausalLM, AutoTokenizer

# 登录 Hugging Face
# 登录后会保存状态，之后无需再次执行此命令
# huggingface-cli login

# 创建 Hugging Face Hub 的 API 实例
api = HfApi()

# 创建模型仓库（如果仓库已经存在，可以跳过这一步）
repo_name = "jlpang888/70B-finetune-full"  # 替换为你的用户名和模型名称
api.create_repo(repo_id=repo_name, exist_ok=True)  # exist_ok=True 表示仓库已存在时不会报错

# 加载模型和 tokenizer（假设你已经在本地训练或保存了模型和 tokenizer）

model_path = "finetune_70B_result/models/meta-llama/Meta-Llama-3.1-8B-Instruct/all_train/meta-llama/Meta-Llama-3-70B/lora_merged_full"
# model = AutoModelForCausalLM.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# # 上传模型和 tokenizer 到 Hugging Face Hub
# model.push_to_hub(repo_name)
# tokenizer.push_to_hub(repo_name)

# # 如果你想上传整个文件夹而不是使用 push_to_hub 方法，可以使用 upload_folder
upload_folder(
    folder_path=model_path,  # 模型文件所在的本地路径
    repo_id=repo_name  # 仓库名称

)


Upload 21 LFS files:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0/21 [00:00<?, ?it/s]


CommitInfo(commit_url='https://huggingface.co/jlpang888/70B-finetune-full/commit/f81a521f9b18ecc0480dc2db047e17a260565bdd', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f81a521f9b18ecc0480dc2db047e17a260565bdd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import os
from huggingface_hub import HfApi, upload_folder
from concurrent.futures import ThreadPoolExecutor, as_completed

# 初始化 Hugging Face Hub 的 API 实例
api = HfApi()

# 仓库名称（整个模型将上传到这个仓库）
repo_name = "jlpang888/70B-finetune-gpt"

# labeling_model="meta-llama/Meta-Llama-3.1-8B-Instruct"
labeling_model="gpt-4o-mini"
# labeling_model="mistralai/Mistral-7B-Instruct-v0.3"

# 定义模型文件路径（包含多个子文件夹，每个子文件夹是模型的一部分）
parent_model_path = f"finetune_70B_result/models/{labeling_model}/all_train/meta-llama/Meta-Llama-3-70B"

# 获取所有子文件夹
subfolders = [f.path for f in os.scandir(parent_model_path) if f.is_dir()]

# 定义并行处理的子文件夹上传任务
def process_subfolder(folder_path):
    folder_name = os.path.basename(folder_path)
    # 可以在这里对每个子文件夹做一些处理，或者准备它们上传
    print(f"Processing folder: {folder_name}")
    # 返回处理结果（这里不上传单个子文件夹，只是并行处理后，最终整合上传）
    return folder_name

# 使用线程池并行处理所有子文件夹
with ThreadPoolExecutor() as executor:
    # 提交任务给线程池
    futures = [executor.submit(process_subfolder, folder) for folder in subfolders]

    # 处理并行执行的结果
    for future in as_completed(futures):
        try:
            result = future.result()
            print(f"Processed {result}")
        except Exception as e:
            print(f"Error processing folder: {e}")

# 并行处理完子文件夹后，上传整个父文件夹到 Hugging Face Hub
print(f"Uploading the full model folder: {parent_model_path} to Hugging Face Hub")

# 上传整个父文件夹到 Hugging Face Hub 仓库
api.create_repo(repo_id=repo_name, exist_ok=True)
upload_folder(
    folder_path=parent_model_path,  # 上传整个父文件夹
    repo_id=repo_name  # Hugging Face 仓库名称
)

print("完整模型文件夹已成功上传到 Hugging Face Hub")
